In [1]:
import base64
import datetime
import json
import os
import random
import re
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from collections import Counter

# Load the Data

In [3]:
d = pd.read_json('tmp/raw_data.json')
d.index.name = 'id'
d = d[['description']]
print len(d)

124011


In [4]:
d.dtypes

description    object
dtype: object

# Some Examples

In [5]:
# for desc in d.description.tolist()[:50]:
#     print desc
#     print '=' * 100

# Basic Count Stats

In [6]:
def count_string_features(s):
    stats = {}
    
    stats['desc_num_breaks'] = s.count('<br />') + s.count('<br/>')
    s = s.replace('<br />', ' ')
    s = s.replace('<br/>', ' ')
    
    stats['desc_num_paragraphs'] = s.count('<p>')
    s = s.replace('<p>', ' ')
    
    stats['desc_num_website_redacted'] = s.count('<a  website_redacted')
    s = s.replace('<a  website_redacted', ' ')
    
    stats['desc_num_emails'] = s.count('kagglemanager@renthop.com')
    s = s.replace('kagglemanager@renthop.com', ' ')
    
    phone_regex = '\d+(\s*-\s*\d+){1,4}'
    stats['desc_num_phones'] = len(re.findall(phone_regex, s))
    s = re.sub(phone_regex, ' ', s)
    
    s = re.sub('\W', ' ', s)
    s = re.sub('\s+', ' ', s)
    stats['desc_num_words'] = len(s.split(' '))
    
    stats['desc_clean'] = s.lower()
    
    return pd.Series(stats)
d = d.description.apply(count_string_features)

In [7]:
noise = np.random.randn(len(d))
d['desc_num_words_q100'] = pd.qcut((d.desc_num_words + 0.1*noise), 100, [0.01 * i for i in range(0, 100)])

# Most Frequent Words

In [8]:
flat_words = [w for desc in d.desc_clean.tolist() for w in desc.split(' ')]
flat_word_counts = Counter(flat_words)
flat_word_counts.most_common(10)

[(u'and', 376446),
 (u'the', 303398),
 (u'a', 272015),
 (u'to', 196570),
 (u'in', 192782),
 (u'with', 175049),
 (u'of', 160566),
 (u'', 139531),
 (u'is', 116454),
 (u'this', 113211)]

In [9]:
top_100_words = set([wc[0] for wc in flat_word_counts.most_common(100)])
top_500_words = set([wc[0] for wc in flat_word_counts.most_common(500)])

In [10]:
d['desc_top_100_word_coverage'] = d.desc_clean.apply(
    lambda ws: len(top_100_words.intersection(set(ws.split(' '))))/100.0)
d['desc_top_500_word_coverage'] = d.desc_clean.apply(
    lambda ws: len(top_500_words.intersection(set(ws.split(' '))))/500.0)
d['desc_top_100_word_ratio'] = d.desc_clean.apply(
    lambda ws: len(top_100_words.intersection(set(ws.split(' '))))/(1.0 * len(ws.split(' '))) if len(ws.split(' ')) > 0 else 0)
d['desc_top_500_word_ratio'] = d.desc_clean.apply(
    lambda ws: len(top_500_words.intersection(set(ws.split(' '))))/(1.0 * len(ws.split(' '))) if len(ws.split(' ')) > 0 else 0)

In [11]:
d.head(20).T

id,0,1,10,100,1000,10000,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,10001,100010,100011,100012
desc_clean,large with awesome terrace accessible via bedr...,prime soho between bleecker and houston newly ...,a brand new 3 bedroom 1 5 bath apartmentenjoy ...,new york chic has reached a new level at 101 ...,step into this fantastic new construction in t...,,take a stroll in central park enjoy the enter...,in a boutique prewar building with 24 hour doo...,why pay other brokers 15 for a great apartment...,spectacular renovated studio in restored town...,top top west village location beautiful pre wa...,beautiful 2br apartment with cherry hardwood f...,hardwood floors exposed brick large living spa...,building amenities garage garden fitness room ...,,midtown neighborhood convenient to grand centr...,newly renovated no fee apt in gramercy with wa...,great value for this 3 bedroom in this area ap...,amazing true three bedroom located in the low ...,naturally illuminated trough large windows wit...
desc_num_breaks,6,0,6,22,6,0,6,10,6,6,10,6,14,18,0,6,0,0,0,6
desc_num_emails,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
desc_num_paragraphs,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,0,0,0,1
desc_num_phones,0,0,1,0,1,0,1,1,2,1,0,0,0,0,0,1,0,0,0,0
desc_num_website_redacted,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,0,0,0,1
desc_num_words,85,38,92,137,199,2,187,77,98,40,103,43,43,58,2,61,110,39,19,206
desc_num_words_q100,0.53,0.18,0.58,0.82,0.94,0.03,0.93,0.47,0.63,0.19,0.66,0.21,0.21,0.32,0.07,0.34,0.69,0.19,0.1,0.95
desc_top_100_word_coverage,0.29,0.15,0.3,0.41,0.39,0.01,0.45,0.31,0.33,0.23,0.46,0.19,0.24,0.3,0.01,0.2,0.42,0.2,0.1,0.43
desc_top_500_word_coverage,0.106,0.06,0.12,0.138,0.176,0.002,0.198,0.078,0.13,0.062,0.16,0.066,0.08,0.102,0.002,0.078,0.142,0.054,0.032,0.226


# Save the Data

In [12]:
del d['desc_clean']

In [13]:
d.to_csv('tmp/features_description.csv')